In [1]:
import tensorflow as tf
from keras.layers import Dense, Conv2D, BatchNormalization, MaxPooling2D, Input, Flatten, Dropout
import numpy as np
import matplotlib.pyplot as plt
# import opendatasets as od
import os
import cv2
from sklearn.preprocessing import LabelBinarizer
import keras
from keras.models import Sequential, Model
from keras.callbacks import EarlyStopping
import random
from keras.applications import EfficientNetB0

# Load and Process Data

Notes about data:

1) the file PARAKETT AUKLET had a few typos. In the Train and Test sets it had two spaces inbetween words, but only one in the Valid set. It should also be PARAKEET. I manually updated this in my version of the files. 

2) The valid and test datasets only have 5 birds images for each bird

In [ ]:
# od.download('https://www.kaggle.com/datasets/gpiosenka/100-bird-species/data')

In [2]:
# Set file path for loading data
train_dir = 'C:/Users/bcbot/COMP_4531/Final/100-bird-species/train'
val_dir = "C:/Users/bcbot/COMP_4531/Final/100-bird-species/valid"
test_dir = "C:/Users/bcbot/COMP_4531/Final/100-bird-species/test"

train_sav = "C:/Users/bcbot/Deep-Learning/birds/train"
val_sav = "C:/Users/bcbot/Deep-Learning/birds/valid"
test_sav = "C:/Users/bcbot/Deep-Learning/birds/test"


In [3]:
# This code just randomly selects some number of birds (specificed by the num_birds varaible). 
# This can be adjusted to speed up training, but need to make sure the softmax layer of the model matches the num_birds

num_birds = 50

birds = [i for i in os.listdir(train_dir)]

random.seed(7284)
rand_idxes = random.sample(range(0, len(birds)), num_birds)

include = [birds[i] for i in rand_idxes]

len(include)

50

In [5]:
# Instantiate the containers for holding image and label data
train_data = []
val_data = []
test_data = []

# Load the first 100 files for each bird for training set
for i in include:
    count = 0
    sub_directory = os.path.join(train_dir, i)

    # # Code to create local directory
    # sav_directory = os.path.join(train_sav, i)
    # os.mkdir(sav_directory)

    for j in os.listdir(sub_directory):
        count += 1
        if count > 100:
            break
        img = cv2.imread(os.path.join(sub_directory, j))
        train_data.append([img, i])

        # # Code to save images locally
        # filename = os.path.join(sav_directory, j)
        # cv2.imwrite(filename, img)



# Load images for validation set
for i in include:
    sub_directory = os.path.join(val_dir, i)

    # # Code to create local directory
    # sav_directory = os.path.join(val_sav, i)
    # os.mkdir(sav_directory)

    for j in os.listdir(sub_directory):
        img = cv2.imread(os.path.join(sub_directory, j))
        val_data.append([img, i])

        # # Code to save images locally
        # filename = os.path.join(sav_directory, j)
        # cv2.imwrite(filename, img)

#Load images for test set
for i in include:
    sub_directory = os.path.join(test_dir, i)

    # # Code to create local directory
    # sav_directory = os.path.join(test_sav, i)
    # os.mkdir(sav_directory)

    for j in os.listdir(sub_directory):
        img = cv2.imread(os.path.join(sub_directory, j))
        test_data.append([img, i])

        # # Code to save images locally
        # filename = os.path.join(sav_directory, j)
        # cv2.imwrite(filename, img)

print(len(train_data))
print(len(val_data))
print(len(test_data))

5000
250
250


In [ ]:
# Check if all images are the same shape
# NOTE: In this particular set, all the iamges are (224, 224, 3), but when I loaded files from all 525 birds there were some that were not, 
# and those screw up the model. So we may need to reshape if we use more birds

count_t = 0
count_v = 0

for t in train_data:
    if t[0].shape != (224, 224, 3):
        count_t += 1
    
for v in val_data:
    if v[0].shape != (224, 224, 3):
        count_v += 1

print(f'There are {count_t} images of different shape in the train data.\nThere are {count_v} images of different shape in the validation data.')
    

In [ ]:
# Reshape images of the wrong size if needed

if count_t != 0:
    for x in train_data:
        if x[0].shape != (224, 224, 3):
            x[0] = cv2.resize(x[0], (224, 224))

if count_v != 0:
    for x in val_data:
        if x[0].shape != (224, 224, 3):
            x[0] = cv2.resize(x[0], (224, 224))


In [ ]:
# Shuffle data

np.random.seed(7284)
np.random.shuffle(train_data)
np.random.shuffle(val_data)

In [ ]:
# Preprocess training data and validation data
lb = LabelBinarizer()

X_train = []
y_train = []
for x, y in train_data:
    X_train.append(x)
    y_train.append(y)

X_train = np.array(X_train)
y_train = np.array(y_train)

y_train_vect = lb.fit_transform(y_train)

X_val = []
y_val = []
for x, y in val_data:
    X_val.append(x)
    y_val.append(y)

X_val = np.array(X_val)
y_val = np.array(y_val)

y_val_vect = lb.fit_transform(y_val)

# Load Model

I initally tried the pre-trained model, but I don't know how to change the number of outputs in the last layer, and trying to run with all 525 species was taking forever. I put some notes on that run but I think I'm just going to give up on trying to use that one. 

In [ ]:
# Set early stopping parameters
callback = EarlyStopping(monitor= 'val_accuracy', patience= 5, start_from_epoch= 5)

Notes for attempt with all 525 birds on pre-trained model:

I tried running this and it earlystopped at 11 epochs. The val_Accuracy never got better than .12 (on epoch 6) but pretty much stayed in the < .01 range. Still not sure how you're supposed to use this model.

In [ ]:
# # Load pre-trained model provided by dataset authors

# model_path = 'C:/Users/bcbot/COMP_4531/Final/100-bird-species/EfficientNetB0-525-(224 X 224)- 98.97.h5'
# model= keras.models.load_model(model_path, custom_objects={'F1_score':'F1_score'})

# layers = model.layers
# for layer in layers:
#     layer.trainable = False

In [ ]:
# #Removing the output layer = model.layers[-2].output
# new_output = model.layers[-2].output
# new_layer = Dense(num_birds, activation='relu')(new_output)

# # creating a new output layer w/ 12 nodes for the subset of data I chose
# model = Model(inputs=model.input, outputs= new_layer)


# model.compile(optimizer ='adam',
#                loss = 'categorical_crossentropy',
#                metrics = ['accuracy'])

In [ ]:
# Check performance of model on training data to evaluate provided weights

# history = model.fit(x = X_train, y= y_train_vect, batch_size=32, validation_data= (X_val, y_val_vect), verbose = 1, epochs = 30, callbacks = [callback])

# Baseline model

A note on this. The EfficientNet model includes a image processing step that expects the pixels to be in the 0-255 range, I had originally been normalizing my pixels by dividing by 255 and that was resulting in terrible results (like 2% accuracy), as soon as I removed the normalization the accuracy jumped up to 80%. 

First test - 100 Birds from 50 species

Each epoch took about 2:30 minutes with a batch size of 80. I let it run for about 25 minute (11 epochs) and I was hovering around 85% accuracy. I ran it again after I froze the first 100 layers, it cut the time per epoch in half and accuracy didn't seem to suffer, I let that one run for 6 epochs and got accuracy at 85+ for all of them. I tried running it again with all of the layers frozen (except the ones I added) and that cut training time per epoch in half again (only 30s per epoch) and I got better accuracy results than with any of the runs. I let that one run until it earlystopped at epoch 11 and none of my accuracy results were less than 92%.

In [ ]:
# Try an Elasticnet model for baseline

# Load Model 
enet = EfficientNetB0(include_top= False, input_shape=(224, 224, 3))

layers = enet.layers

for layer in layers:
    layer.trainable = False

# Add a flatten and softmax layer
model= Sequential([
    enet,
    Flatten(),
    Dense(num_birds, activation='softmax')
])

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(x = X_train, y= y_train_vect, batch_size=80, validation_data= (X_val, y_val_vect), verbose = 1, epochs = 30, callbacks = [callback])

Notes for attempt with all 525 birds:

I manually stopped this one after it was running for about 2.5 hours with a batch size of 32. This one never got great accuracy, but was pretty consistently performing between .12 and .14 over 18 epochs. So it's already performing better than the provided model, which is just confusing me more. 

Notes for 100 birds from 50 species:

This took about 20 seconds per epoch with a batch size of 80. Early stopping kicked in at epcoh 13. The model pretty much just hung out at a val accuracy of .60ish for all the epochs. 

In [ ]:
# Try a very simple model to see how it performs
model = Sequential([
    Conv2D(32, 3, input_shape = (224, 224, 3), activation= 'relu'),
    MaxPooling2D(),
    Dropout(.2),
    Flatten(),
    Dense(num_birds, activation='softmax')
])

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history2 = model.fit(x = X_train/255, y= y_train_vect, batch_size=80, validation_data= (X_val/255, y_val_vect), verbose = 1, epochs = 30, callbacks = [callback])